In [ ]:
!pip install catboost

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
train = pd.read_csv('/content/sample_data/contest_train.csv')
train.head()

,ID,TARGET,FEATURE_0,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,FEATURE_8,FEATURE_9,FEATURE_10,FEATURE_11,FEATURE_12,FEATURE_13,FEATURE_14,FEATURE_15,FEATURE_16,FEATURE_17,FEATURE_18,FEATURE_19,FEATURE_20,FEATURE_21,FEATURE_22,FEATURE_23,FEATURE_24,FEATURE_25,FEATURE_26,FEATURE_27,FEATURE_28,FEATURE_29,FEATURE_30,FEATURE_31,FEATURE_32,FEATURE_33,FEATURE_34,FEATURE_35,FEATURE_36,FEATURE_37,...,FEATURE_220,FEATURE_221,FEATURE_222,FEATURE_223,FEATURE_224,FEATURE_225,FEATURE_226,FEATURE_227,FEATURE_228,FEATURE_229,FEATURE_230,FEATURE_231,FEATURE_232,FEATURE_233,FEATURE_234,FEATURE_235,FEATURE_236,FEATURE_237,FEATURE_238,FEATURE_239,FEATURE_240,FEATURE_241,FEATURE_242,FEATURE_243,FEATURE_244,FEATURE_245,FEATURE_246,FEATURE_247,FEATURE_248,FEATURE_249,FEATURE_250,FEATURE_251,FEATURE_252,FEATURE_253,FEATURE_254,FEATURE_255,FEATURE_256,FEATURE_257,FEATURE_258,FEATURE_259
0,1053371037,1,2.0,2.0,0.0,0.0,0.0,0.0,0.0,-114.527812,-17.218055,0.0,0.0,0.0,25.257841,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.492541,0.0,3.770712,-2.165145,0.0,-11.508692,0.0,0.0,0.0,0.0,0.0,0.0,0.709990,-166.128241,16.497987,-109.953267,-10.257451,...,2.0,22.0,56.0,-18.199109,1.0,433.147718,4.0,52.0,55.724127,0.0,33.779127,0.0,48.382771,-6.547478,0.0,62.754784,111.843533,213.048521,20.0,18.0,225.386422,0.0,0.0,0.0,160.783415,0.0,2.0,0.0,19.0,0.0,39.0,2.0,223.118594,290.015143,1.0,1.0,0.0,1.0,2.0,2.0
1,76507184,0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,329.828334,NaN,3.0,2.0,1.0,767.317882,30.0,11.0,1.0,0.0,0.0,1.0,0.0,0.0,-1.779101,0.0,-27.718626,-2.046630,0.0,-48.533875,0.0,0.0,0.0,0.0,0.0,0.0,4198.421679,6935.032043,3326.119168,4750.740829,8171.976929,...,1.0,288.0,540.0,1573.705923,1.0,885.756703,104.0,230.0,349.871581,0.0,6367.475792,0.0,106.445674,6535.301751,0.0,429.122711,325.660562,720.583426,361.0,135.0,911.831620,0.0,81.0,1.0,485.233915,0.0,0.0,0.0,1.0,0.0,25.0,82.0,479.616045,611.651594,0.0,1.0,0.0,1.0,1.0,1.0
2,76753690,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.427338,-6.680747,0.0,0.0,0.0,1453.022898,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.042108,0.0,-0.090641,-1.499671,0.0,-6.227192,0.0,0.0,0.0,0.0,0.0,0.0,-89.245457,9.436363,0.792981,61.085481,4.367151,...,2.0,92.0,161.0,2.543405,1.0,504.081602,13.0,89.0,81.637348,0.0,13.527245,0.0,50.178073,29.679975,0.0,228.212833,173.897261,395.110199,28.0,15.0,343.967505,0.0,0.0,0.0,67.814125,0.0,0.0,0.0,78.0,0.0,119.0,0.0,352.268014,564.573421,1.0,1.0,0.0,1.0,1.0,1.0
3,1026167358,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,23.621397,36.659534,0.0,0.0,0.0,31.190091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.717799,0.0,4.728385,1.393742,0.0,97.362996,0.0,0.0,0.0,0.0,0.0,0.0,31.692737,-13.471019,-28.444859,18.306662,11.252959,...,2.0,66.0,95.0,74.923704,3.0,610.918525,0.0,5.0,15.496008,0.0,77.928861,0.0,-68.997270,-7.829144,0.0,86.471577,74.197509,129.144486,25.0,12.0,437.126266,0.0,0.0,0.0,-33.449029,0.0,0.0,0.0,0.0,0.0,1.0,0.0,410.524985,421.653876,1.0,0.0,0.0,1.0,2.0,2.0
4,7220095746,0,4.0,3.0,1.0,0.0,1.0,0.0,0.0,677.745861,NaN,3.0,2.0,1.0,525.617364,3.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,-11.515780,0.0,-3.293282,-12.724665,0.0,32.755535,0.0,0.0,0.0,0.0,0.0,0.0,903.748860,1979.278641,10935.990056,3591.990400,3421.519324,...,1.0,7.0,21.0,411.043807,8.0,587.200844,5.0,85.0,88.260674,0.0,3346.536049,0.0,166.360435,3489.936805,0.0,-41.518356,45.664366,60.847476,9.0,47.0,251.658361,0.0,0.0,0.0,427.417631,0.0,0.0,0.0,0.0,0.0,21.0,0.0,439.171356,476.189288,1.0,1.0,0.0,1.0,1.0,1.0


In [22]:
#несбалансированные классы
train.TARGET.value_counts()

0    17372
1     5650
2     1499
Name: TARGET, dtype: int64

In [23]:
#Undersampling+замена пропусков на среднее+удвоение выборки
train = pd.concat([train, train], axis=0)
y = train['TARGET']
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.33, random_state=42, shuffle=True)
X_train = X_train.apply(lambda x: x.fillna(x.mean()),axis = 1)
X_test = X_test.apply(lambda x: x.fillna(x.mean()),axis = 1)
X_train = X_train.sort_values(by='TARGET').reset_index(drop=True).drop('ID', axis=1)
rand_ind = np.random.choice(23283, 23283//2)
X_train = X_train.sample(frac=1).reset_index(drop=True)
X_train = X_train.drop(rand_ind,axis = 0)
y_train = X_train.TARGET
X_train = X_train.drop('TARGET', axis=1)
X_test = X_test.drop('TARGET', axis=1).drop('ID', axis=1)

In [24]:
#Для поиска выбросов используем Изоляционный лес
from sklearn.ensemble import IsolationForest
clf = IsolationForest(max_samples=100, random_state = 1, contamination= 'auto')
preds = clf.fit_predict(X_train)
X_train['anomaly'] = preds
X_train['Target'] = y_train
X_train = X_train.loc[X_train['anomaly'] != -1]
y_train = X_train['Target']
X_train = X_train.drop(['anomaly','Target'],axis = 1)

In [ ]:
#Используем рандомфорест с сбалансированием классов
rf = RandomForestClassifier(n_estimators = 500,random_state = 242,class_weight = 'balanced')
params = {'max_depth': range(1,100,10), 'min_samples_leaf': range(20,150,10), 'min_samples_split': range(20,150,10)}
clf_rf = RandomizedSearchCV(rf, params, scoring = 'f1_macro', cv = 5, n_jobs = -1)
clf_rf.fit(X_train,y_train)

In [ ]:
print('метрика на кросс-валидации', clf_rf.best_score_)
best_clf = clf_rf.best_estimator_
f1_score(y_test, best_clf. predict(X_test), average='macro')
#получили недостаточно(0.52)

In [29]:
params = { 
    "iterations": 500, 
    "learning_rate": 0.1, 
    "depth": 10, 
    "min_data_in_leaf": 20,
    "silent" : True
}

cd = CatBoostClassifier(**params)

In [ ]:
cross_val_score(cd, X_train, y_train, scoring='f1_macro', cv=5)

In [30]:
#Уже лучше(0.75)
cd.fit(X_train,y_train)
f1_score(y_test, cd.predict(X_test), average='macro')

0.7527788266003271

In [74]:
#Получили таблицу с таргетом для теста
test = pd.read_csv('/content/sample_data/contest_test.csv')
predict = cd.predict(test.drop('ID',axis = 1))
pred_ar = []
for i in range(predict.shape[0]):
    pred_ar.append(int(predict[i,0]))
dt = {'ID': np.array(test.ID), 'TARGET': np.array(pred_ar).T}
pred_df = pd.DataFrame(data = dt)
pred_df.to_csv('contest_answer.csv')

In [76]:
#Получили таблицу с вероятностями предсказания классов, сортирую ее и вывожу то что нужно
pred_proba = cd.predict(test.drop('ID',axis = 1),prediction_type='Probability')

In [92]:
pred_df_proba = pred_df.copy()
pred_df_proba['Probability'] = np.amax(pred_proba,axis = 1)
pred_df_1 = pred_df_proba[(pred_df_proba['TARGET']==1)].iloc[:500,:].sort_values(by='Probability', ascending=False)
pred_df_2 = pred_df_proba[(pred_df_proba['TARGET']==2)].iloc[:200,:].sort_values(by='Probability', ascending=False)

In [99]:
segm_df = pd.concat([pred_df_1,pred_df_2]).drop('Probability', axis =1)
segm_df.to_csv('contest_segments.csv')